In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd

In [2]:
# parameters
initial_balance = 100
max_months = 61 # we're going to look at the beginning of month 61 to evaluate the end of month 60
crops_data = pd.read_csv("crops.csv")
crops_data.index = crops_data['crop']
crops_data

,crop,cost,selling price,time to grow
crop,,,,
peas,peas,10,12,2
potatos,potatos,5,8,4
raspberries,raspberries,15,25,4
saffron,saffron,175,500,6


In [12]:
a = [0] * growth_time + [1] * (max_months - growth_time)
len(a)

61

In [15]:
a = [1, 2, 3, 4]
a[:2] = [0]*2
a

[0, 0, 3, 4]

In [19]:
len(peas_constraint)

3721

In [21]:
# goal is to maximize profit by only buying crops

# the real variables you can control
peas = cp.Variable((max_months), "peas") # number of peas to buy at the beginning of month i
potatos = cp.Variable((max_months), "potatos") # number of potatos to buy at the beginning of month i
raspberries = cp.Variable((max_months), "raspberries") # number of raspberries to buy at the beginning of month i
saffron = cp.Variable((max_months), "saffron") # number of saffron to buy at the beginning of month i

# auxilliary variable
grown_peas = cp.Variable((max_months), "grown_peas") # number of peas that have grown at the beginning of month i
grown_potatos = cp.Variable((max_months), "grown_potatos") # number of potatos that have grown at the beginning of month i
grown_raspberries = cp.Variable((max_months), "grown_raspberries") # number of raspberries that have grown at the beginning of month i
grown_saffron = cp.Variable((max_months), "grown_saffron") # number of saffron that have grown at the beginning of month i

# technically auxilliary variable, since can be written in terms of the variables you can control, but will optimize the last balance
balance = cp.Variable((max_months), "balance") # the balance at the beginning of each month after buying crops

# non-negativity constraints
constraints = [
    peas >= 0,
    potatos >= 0,
    raspberries >= 0,
    saffron >= 0,
    grown_peas >= 0,
    grown_potatos >= 0,
    grown_raspberries >= 0,
    grown_saffron >= 0,
    balance >= 0
]

# grown peas
growth_time = crops_data['time to grow']['peas']
peas_constraint = [peas[i - growth_time] for i in range(max_months)]
peas_constraint[:growth_time] = [0] * growth_time
for i in range(len(peas_constraint)):
    constraints.append(grown_peas[i] == peas_constraint[i])

# grown potatos
growth_time = crops_data['time to grow']['potatos']
potatos_constraint = [potatos[i - growth_time] for i in range(max_months)]
potatos_constraint[:growth_time] = [0] * growth_time
for i in range(len(potatos_constraint)):
    constraints.append(grown_potatos[i] == potatos_constraint[i])

# grown raspberries
growth_time = crops_data['time to grow']['raspberries']
raspberries_constraint = [raspberries[i - growth_time] for i in range(max_months)]
raspberries_constraint[:growth_time] = [0] * growth_time
for i in range(len(raspberries_constraint)):
    constraints.append(grown_raspberries[i] == raspberries_constraint[i])

# grown saffron
growth_time = crops_data['time to grow']['saffron']
saffron_constraint = [saffron[i - growth_time] for i in range(max_months)]
saffron_constraint[:growth_time] = [0] * growth_time
for i in range(len(saffron_constraint)):
    constraints.append(grown_saffron[i] == saffron_constraint[i])

# balance constraints
constraints.append(balance[0] == initial_balance - (10*peas[0] + 5*potatos[0] + 15*raspberries[0] + 175*saffron[0]))

for i in range(1, max_months):
    cost = 10*peas[i] + 5*potatos[i] + 15*raspberries[i] + 175*saffron[i]
    revenue = 12*grown_peas[i] + 8*grown_potatos[i] + 25*grown_raspberries[i] + 500*grown_saffron[i]
    constraints.append(balance[i] == balance[i-1] - cost + revenue)

# max balance[max_months]
obj = cp.Maximize(balance[max_months - 1])

prob = cp.Problem(obj, constraints)
optimum = prob.solve(solver=cp.CLARABEL)

print("optimal value: %s\n" % optimum)
for variable in prob.variables():
    print("Variable %s: value %s" % (variable.name(), variable.value))

optimal value: 3625086.0750254076

Variable balance: value [ 4.25858533e-05 -7.58202469e-06  4.52705435e-06 -6.42161385e-06
  1.86489699e-05 -8.08038935e-06  1.06301756e-04  3.87365048e-06
  3.80012560e-05  6.80706379e-06  7.04633576e-05  3.75396090e-06
  2.34120762e-04  3.61413634e-05  1.26694301e-04  4.45340501e-05
  1.88054554e-04  4.13555311e-05  5.01496706e-04  1.27144172e-04
  3.52037279e-04  1.56201594e-04  4.86783303e-04  1.70647524e-04
  1.05544395e-03  3.95445759e-04  9.24774426e-04  5.05309981e-04
  1.27408580e-03  5.92918349e-04  2.27882194e-03  1.19992483e-03
  2.38874152e-03  1.59709915e-03  3.35229059e-03  2.06982507e-03
  4.98002789e-03  3.63508205e-03  5.99936320e-03  4.93532124e-03
  8.68372510e-03  6.97634481e-03  1.06506065e-02  1.05496878e-02
  1.48619001e-02  1.40334847e-02  2.35715734e-02  2.06774309e-02
  2.29976120e-02  2.69889933e-02  4.20776144e-02  3.43481445e-02
  8.58767922e-02  1.50581090e-02  5.85873495e-02  8.97878954e-02
  1.62586183e-01  3.24747971e-0

In [4]:
print(type(balance[i] == 2))

<class 'cvxpy.constraints.zero.Equality'>
